In [3]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [7]:
from SourceCodeTools.models.training_config import get_config, save_config, load_config
from SourceCodeTools.code.data.dataset.Dataset import SourceGraphDataset
from SourceCodeTools.models.graph.train.sampling_multitask2 import SamplingMultitaskTrainer, \
    select_device
from SourceCodeTools.models.graph.train.objectives.NodeClassificationObjective import ClassifierTargetMapper
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphClassifierObjective
from SourceCodeTools.models.graph.train.utils import get_name, get_model_base
from SourceCodeTools.models.graph import RGGAN, RGCN
from SourceCodeTools.code.data.file_utils import unpersist

import json
from copy import copy

import pandas as pd
from tqdm import tqdm

from torch import nn
from datetime import datetime
from os.path import join

# Prepare parameters and options

Full list of options that can be added can be found in `SourceCodeTools/models/training_options.py`. They are ment to be used as arguments for cli trainer. Trainer script can be found in `SourceCodeTools/scripts/train.py`.

For the task of subgraph classification the important options are:
- `subgraph_partition` is path to subgraph-based train/val/test sets. Storead as Dataframe with subgraph id and partition mask
- `subgraph_id_column` is a column is `common_edges` file that stores subgraph id.
- For variable misuse task (same will apply to authorship attribution) subgraphs are created for individual functions (files for SCAA). The label is stored in `common_filecontent`.

In [8]:
tokenizer_path = "sentencepiece_bpe.model"

data_path = "variable_misuse_graph_2_percent_balanced/with_ast"
subgraph_partition = join(data_path, "partition.json.bz2")
filecontent_path = join(data_path, "common_filecontent.json.bz2")
checkpoint_path = None

In [10]:
# config = get_config(
#     subgraph_id_column="file_id",
#     data_path=data_path,
#     model_output_dir=data_path,
#     partition=subgraph_partition,
#     tokenizer_path=tokenizer_path,
#     objectives="subgraph_clf",
#     batch_size=1,
# )

config = load_config(join(checkpoint_path, "config.yaml"))
config["TRAINING"]["restore_state"] = True
config["TRAINING"]["batch_size"] = 1

# Load dataset

In [11]:
dataset = SourceGraphDataset(
    **{**config["DATASET"], **config["TOKENIZER"]}
)
ntypes, etypes = dataset.get_graph_types()
config["TRAINING"]['ntypes'] = ntypes
config["TRAINING"]['etypes'] = etypes

# Declare target loading function (labels)

In [17]:
from IPython.display import Markdown
from ast import literal_eval

class SubgraphClassifierObjectiveWithSaliency(SubgraphClassifierObjective):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def saliency_for_batch(
            self, batch_ind, batch, optimizer
    ):
        optimizer.zero_grad()
        
        outputs = self(
            **batch
        )

        # collect gradients
        outputs.gnn_output.node_embeddings["node_"].retain_grad()
        outputs.gnn_output.input_embeddings["node_"].retain_grad()
        outputs.loss.backward()

        # get input node ids
        def get_original_node_ids(block):
            return block.srcnodes["node_"].data["original_id"].tolist()

        original_node_ids = get_original_node_ids(batch["blocks"])
        graph_node_ids = get_original_node_ids(batch["blocks"])

        # compute saliency scores for nodes
        def compute_saliency_score(tensor_):
            return tensor_.abs().mean(dim=-1).numpy()

        input_node_saliency = compute_saliency_score(
            outputs.gnn_output.input_embeddings["node_"].grad.data
        )
        output_node_saliency = compute_saliency_score(
            outputs.gnn_output.node_embeddings["node_"].grad.data
        )

        def get_source_code_and_misuse_span(fn_id):
            text = self.file_information[fn_id]["filecontent"]
            misuse_span = literal_eval(self.file_information[fn_id]["misuse_span"])
            return text, misuse_span

        code_text, misuse_span = get_source_code_and_misuse_span(batch["indices"][0])

        # retrieve additional information for nodes for display
        def get_information_for_nodes(node_ids):
            node_info = dataset._graph_storage.database.query(f"""
            select nodes.id, name, string, type_desc as type from nodes
            left join node_strings on node_strings.id = nodes.id
            left join node_types on node_types.type_id = nodes.type
            where nodes.id in ({",".join(set(map(str, node_ids)))})
            """)

            return {
                "names": dict(zip(node_info["id"], node_info["name"])),
                "strings": dict(zip(node_info["id"], node_info["string"])),
                "types": dict(zip(node_info["id"], node_info["type"]))
            }

        node_info = get_information_for_nodes(original_node_ids + graph_node_ids)

        def make_saliency_summary_table(node_ids, saliency, node_info):
            # sort input nodes by saliency
            id_saliency = sorted(
                zip(node_ids, saliency),
                key=lambda x: x[1], reverse=True
            )

            input_nodes_summary = {
                "ids": list(map(lambda x: x[0], id_saliency)),
                "score": list(map(lambda x: x[1], id_saliency)),
                "name": list(map(lambda x: node_info["names"][x[0]], id_saliency)),
                "type": list(map(lambda x: node_info["types"][x[0]], id_saliency)),
                "string": list(map(lambda x: node_info["strings"].get(x[0], pd.NA), id_saliency)),
            }

            return pd.DataFrame(input_nodes_summary)

        input_node_saliency_summary = make_saliency_summary_table(original_node_ids, input_node_saliency, node_info)
        output_node_saliency_summary = make_saliency_summary_table(graph_node_ids, output_node_saliency, node_info)

        def print_code(code: str, misuse):
            code = code.lstrip()
            if len(misuse) == 0:
                print(code)
            else:
                display(Markdown(
                    code[:misuse[0]].replace(" ","&nbsp;") + f'<span style="color:blue">{code[misuse[0]: misuse[1]]}</span>' + code[misuse[1]: ].replace(" ","&nbsp;")
                ))

        prediction = outputs.prediction[0][0]

        print_code(code_text, misuse_span)
        print(f"Predicted class: {prediction.argmax().item()}")
        print(f"True misuse location: {misuse_span}")
        print("Input saliency")
        print(pd.DataFrame(input_node_saliency_summary)[["ids", "score", "name"]].head(10).to_string())
        print("Node embedding")
        print(pd.DataFrame(output_node_saliency_summary)[["ids", "score", "name"]].head(10).to_string())
        print("\n\n\n")

    def saliency(self, data_split, optimizer):
        self.file_information = {}
        with open(join(data_path, "common_filecontent.json")) as filecontents:
            for line in filecontents:
                entry = json.loads(line)
                self.file_information[entry["id"]] = entry
        
        for batch_ind, batch in enumerate(tqdm(
                self.get_iterator(data_split), total=getattr(self, f"num_{data_split}_batches")
        )):
            self.saliency_for_batch(batch_ind, batch, optimizer)
            if batch_ind == 20:
                break
            

In [18]:
def load_labels():
    filecontent = unpersist(filecontent_path)
    return filecontent[["id", "label"]].rename({"id": "src", "label": "dst"}, axis=1)

In [19]:
class Trainer(SamplingMultitaskTrainer):
    def create_objectives(self, dataset, tokenizer_path):
        self.objectives = nn.ModuleList()
        
        self.objectives.append(
            self._create_subgraph_objective(
                objective_name="MisuseSubgraphClassifierObjective",
                objective_class=SubgraphClassifierObjectiveWithSaliency,
                dataset=dataset,
                tokenizer_path=tokenizer_path,
                labels_fn=load_labels,
                label_loader_class=ClassifierTargetMapper,
                label_loader_params={"emb_size": None, "tokenizer_path": None, "use_ns_groups": False}
            )
        )
        
    def saliency(self):
        objective = self.objectives[0]
        objective.eval()
        objective.saliency("test", self.optimizer)

In [20]:
def compute_saliency(
        dataset, model_name, model_params, trainer_params, model_base_path,
        tokenizer_path=None, trainer=None, load_external_dataset=None
):
    model_params = copy(model_params)
    trainer_params = copy(trainer_params)

    if trainer is None:
        trainer = SamplingMultitaskTrainer

    device = select_device(trainer_params)

    trainer_params['model_base_path'] = model_base_path

    trainer = trainer(
        dataset=dataset,
        model_name=model_name,
        model_params=model_params,
        trainer_params=trainer_params,
        restore=trainer_params["restore_state"],
        device=device,
        pretrained_embeddings_path=trainer_params["pretrained"],
        tokenizer_path=tokenizer_path,
    )

    trainer.saliency()

In [21]:
compute_saliency(
    dataset, 
    model_name=RGCN, 
    model_params=config["MODEL"],
    trainer_params=config["TRAINING"],
    model_base_path=checkpoint_path,
    trainer=Trainer
)

  0%|          | 1/7483 [00:00<1:46:53,  1.17it/s]

def load_user_configuration(config_path):
    'Create a user configuration dictionary from config files\n\n    :param config_path: ``str`` path where the configuration files are kept\n    '
    user_defined_config = dict()
    user_config_file = os.path.join(config_path, 'openstack_user_config.yml')
    if os.path.isfile(user_config_file):
        with open(user_config_file, 'rb') as f:
            user_defined_config.update((yaml.safe_load(f.read()) or {
                
            }))
    base_dir = os.path.join(config_path, 'conf.d')
    if os.path.isdir(base_dir):
        _extra_config(user_defined_config, base_dir)
    if (not user_defined_config):
        raise SystemExit(('No user config loaded\nNo openstack_user_config files are available in either \n%s\nor \n%s/conf.d directory' % (config_path, config_path)))
    return user_defined_config
Predicted class: 0
True misuse location: []
Input saliency
       ids     score                         name
0   573577  0.000029       ar

def&nbsp;echo(request):
&nbsp;&nbsp;&nbsp;&nbsp;try:
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;args&nbsp;=&nbsp;parser.parse(hello_args,&nbsp;request)
&nbsp;&nbsp;&nbsp;&nbsp;except&nbsp;ValidationError&nbsp;as&nbsp;err:
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;json_response(<span style="color:blue">request</span>.messages,&nbsp;status=err.status_code)
&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;json_response(args)

Predicted class: 1
True misuse location: [141, 148]
Input saliency
      ids     score                                            name
0  871114  0.000063                          arg_0x16f92ac20e664063
1  651916  0.000058                       Module_0x16f92ac20e9d49aa
2  492356  0.000058                       Assign_0x16f92ac20eb9e065
3  862369  0.000051                       Return_0x16f92ac20f2a871d
4  572907  0.000049                ExceptHandler_0x16f92ac20f0ad12b
5  840087  0.000049                         Call_0x16f92ac20f88581a
6  235607  0.000049  ValidationError@FunctionDef_0x16f92ac20eaf7963
7  327040  0.000048                         Call_0x16f92ac20eb7f6ea
8  728932  0.000047                                           ▁json
9  379828  0.000047                       Return_0x16f92ac20fa01651
Node embedding
      ids     score                                            name
0    3389  0.000323             args@FunctionDef_0x16f92ac20eaf7963
1   22958  0.000323               

def&nbsp;_argcheck(self,&nbsp;a,&nbsp;b):
&nbsp;&nbsp;&nbsp;&nbsp;self.a&nbsp;=&nbsp;a
&nbsp;&nbsp;&nbsp;&nbsp;self.b&nbsp;=&nbsp;b
&nbsp;&nbsp;&nbsp;&nbsp;self.d&nbsp;=&nbsp;log(((b&nbsp;*&nbsp;1.0)&nbsp;/&nbsp;a))
&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;(((a&nbsp;>&nbsp;0)&nbsp;&&nbsp;(b&nbsp;>&nbsp;0))&nbsp;&&nbsp;(b&nbsp;>&nbsp;<span style="color:blue">b</span>))

...

  0%|          | 19/7483 [00:02<15:13,  8.17it/s]

Predicted class: 1
True misuse location: [64, 68]
Input saliency
      ids     score                                    name
0  936862  0.000113                  arg_0x16f92ac21750c46f
1  162686  0.000113                  arg_0x16f92ac217991f7e
2  297249  0.000102               Module_0x16f92ac217f9b5a5
3  575580  0.000092  removeWriter@Module_0x16f92ac217f9b5a5
4  706125  0.000085     self@FunctionDef_0x16f92ac21759ee95
5  862631  0.000083                     _updateRegistration
6  149886  0.000078            Attribute_0x16f92ac217352510
7  646203  0.000078    EPoll@FunctionDef_0x16f92ac21759ee95
8  547000  0.000078       fd@FunctionDef_0x16f92ac21759ee95
9  408565  0.000078    super@FunctionDef_0x16f92ac21759ee95
Node embedding
      ids     score                            name
0    1135  0.000536         Call_0x16f92ac2176c23c2
1   32704  0.000536                              ▁E
2  118936  0.000536                          Writer
3  120474  0.000536         Call_0x16f92ac2170311f5


def&nbsp;variables(i,&nbsp;o):
&nbsp;&nbsp;&nbsp;&nbsp;'\n&nbsp;&nbsp;&nbsp;&nbsp;WRITEME\n\n&nbsp;&nbsp;&nbsp;&nbsp;Parameters\n&nbsp;&nbsp;&nbsp;&nbsp;----------\n&nbsp;&nbsp;&nbsp;&nbsp;i&nbsp;:&nbsp;list\n&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Input&nbsp;L{Variable}s.\n&nbsp;&nbsp;&nbsp;&nbsp;o&nbsp;:&nbsp;list\n&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Output&nbsp;L{Variable}s.\n\n&nbsp;&nbsp;&nbsp;&nbsp;Returns\n&nbsp;&nbsp;&nbsp;&nbsp;-------\n&nbsp;&nbsp;&nbsp;&nbsp;object\n&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;The&nbsp;set&nbsp;of&nbsp;Variables&nbsp;that&nbsp;are&nbsp;involved&nbsp;in&nbsp;the&nbsp;subgraph&nbsp;that&nbsp;lies\n&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;between&nbsp;i&nbsp;and&nbsp;o.&nbsp;This&nbsp;includes&nbsp;i,&nbsp;o,&nbsp;orphans(i,&nbsp;o)&nbsp;and&nbsp;all&nbsp;values&nbsp;of\n&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;all&nbsp;intermediary&nbsp;steps&nbsp;from&nbsp;i&nbsp;to&nbsp;o.\n\n&nbsp;&nbsp;&nbsp;&nbsp;'
&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;variables_and_orphans(i,&nbsp;<span style="color:blue">i</span>)[0]

  0%|          | 20/7483 [00:02<15:19,  8.12it/s]

Predicted class: 0
True misuse location: [443, 444]
Input saliency
      ids     score                                 name
0  771983  0.000123               arg_0x16f92ac2198e9172
1  389049  0.000123               arg_0x16f92ac219446cc8
2  706169  0.000112            Module_0x16f92ac219b53366
3  964161  0.000093            Return_0x16f92ac219f64422
4  354131  0.000092             Index_0x16f92ac219bbacad
5  953377  0.000090       FunctionDef_0x16f92ac2192aa304
6   90449  0.000088              Call_0x16f92ac2193d6c19
7  822798  0.000085                           ▁variables
8  553937  0.000084  variables@Module_0x16f92ac219b53366
9  956524  0.000083     i@FunctionDef_0x16f92ac2192aa304
Node embedding
      ids    score                                                  name
0   90449  0.00063                               Call_0x16f92ac2193d6c19
1  134562  0.00063                                                    ▁i
2  172444  0.00063                      o@FunctionDef_0x16f92ac2192aa304